# Get Citibike trip data

## Prerequisites
`boto3` uses AWS credentials from `~/.aws/credentials`
* [Create AWS account](https://aws.amazon.com/resources/create-account/)
* [Install AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)
* [Configure AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html)
    * run `aws configure` in terminal and set credentials from access key created for a user

## Citibike AWS S3 bucket
[https://s3.amazonaws.com/tripdata/index.html](https://s3.amazonaws.com/tripdata/index.html)

In [20]:
import boto3
import os, zipfile

In [21]:
TRIPDATA_BUCKET = 'tripdata'
TRIPDATA_DIR_TO = 'data/tripdata/'

s3 = boto3.resource('s3')
tripdata = s3.Bucket(TRIPDATA_BUCKET)

In [22]:
# Download all S3 objects in bucket
for s3_object in tripdata.objects.all():
    _, filename = os.path.split(s3_object.key)
    print(f'Downloading {filename} to {TRIPDATA_DIR_TO}...')
    tripdata.download_file(s3_object.key, TRIPDATA_DIR_TO + filename)

In [30]:
# remove index.html and combo csv file
!rm 'data/tripdata/index.html'
!rm 'data/tripdata/201307-201402-citibike-tripdata.zip'

print(f'{len(os.listdir(TRIPDATA_DIR_TO))} objects downloaded')

rm: data/tripdata/index.html: No such file or directory
183 objects downloaded


In [ ]:
# Unzip all
os.chdir(TRIPDATA_DIR_TO)

for file in os.listdir(TRIPDATA_DIR_TO):
    if zipfile.is_zipfile(file):
        with zipfile.ZipFile(file) as zipitem:
            zipitem.extractall()